In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, ReLU, Softmax
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam


In [11]:
import cv2
import numpy as np


In [12]:
SIZE = 256
N = 1

In [13]:
X = np.zeros((N, SIZE, SIZE))
A = np.zeros((N, SIZE, SIZE))
B = np.zeros((N, SIZE, SIZE))
for i in range(1, N + 1):
    X[i-1] = cv2.imread('data/bnw/' + str(i + 7) + '.jpeg', 0)
    A[i-1] = cv2.imread('data/a/' + str(i + 7) + '.jpeg', 0)
    B[i-1] = cv2.imread('data/b/' + str(i + 7) + '.jpeg', 0)



In [14]:
bins = np.arange(33) * 8
a_binned = np.digitize(A, bins) - 1
b_binned = np.digitize(B, bins) - 1
y_binned = np.stack((a_binned, b_binned)).transpose((1, 2, 3, 0))

In [15]:
X = X.reshape(N, SIZE, SIZE, 1)
y = np.stack((A, B)).transpose((1, 2, 3, 0))

In [16]:
# Normalize the data
X = X / 255
y = y / 255

In [17]:
# inputs = Input((SIZE, SIZE, 1))
# # TODO: add batch normalization
# conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
# conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#
# conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
# conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
# pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#
# conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
# conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
# pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#
# conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
# conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
# drop4 = Dropout(0.5)(conv4)
#
# up5 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size = (2,2))(conv4))
# merge5 = concatenate([conv3, up5], axis = 3)
# conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
# conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
#
# up6 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size = (2,2))(conv5))
# merge6 = concatenate([conv2, up6], axis = 3)
# conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
# conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
#
# up7 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size = (2,2))(conv6))
# merge7 = concatenate([conv1, up7], axis = 3)
# conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
# conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
#
# conv8 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
# conv8 = Conv2D(2, 1, activation='sigmoid')(conv8)
#
# model = Model(inputs, conv8)
# model.summary()
#
# model.compile(optimizer=Adam(learning_rate=0.1), loss='mae')

Following are utility functions for creating a U-net Model

In [18]:
def conv_stack(input_layer, filters):
    conv1 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(input_layer)
    batch_norm1 = BatchNormalization()(conv1)
    relu1 = ReLU()(batch_norm1)

    conv2 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(relu1)
    batch_norm2 = BatchNormalization()(conv2)
    relu2 = ReLU()(batch_norm2)

    return relu2

In [19]:
def encoder_block(input_layer, filters):
    conv = conv_stack(input_layer, filters)
    max_pool = MaxPooling2D(pool_size=(2, 2))(conv)

    return conv, max_pool

In [20]:
def decoder_block(input_layer, skip_layer, filters):
    up = Conv2DTranspose(filters, 2, strides=2, padding='same')(input_layer)
    conc = Concatenate()([up, skip_layer])
    dec = conv_stack(conc, filters)

    return dec

In [21]:
def get_model(size, init_filters):
    inputs = Input((size, size, 1))

    conv1, max_pool1 = encoder_block(inputs, init_filters)
    conv2, max_pool2 = encoder_block(max_pool1, init_filters * 2)
    conv3, max_pool3 = encoder_block(max_pool2, init_filters * 4)

    middle_block = conv_stack(max_pool3, init_filters * 8)

    decoder1 = decoder_block(middle_block, conv3, init_filters * 4)
    decoder2 = decoder_block(decoder1, conv2, init_filters * 2)
    decoder3 = decoder_block(decoder2, conv1, init_filters)

    # soft = Softmax(axis=1)(decoder3)
    output_a = Conv2D(32, 1, padding='same', activation='softmax')(decoder3)
    output_b = Conv2D(32, 1, padding='same', activation='softmax')(decoder3)

    model = Model(inputs, [output_a, output_b])
    return model

In [22]:
model = get_model(SIZE, 64)
model.summary()
model.compile(optimizer=Adam(0.01), loss='sparse_categorical_crossentropy', metrics='accuracy')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 256, 256, 64) 0           batch_normalization[0][0]        
_____________

TypeError: Type of `metrics` argument not understood. Expected a list or dictionary, found: accuracy

In [23]:
model.fit(X, [a_binned, b_binned], epochs=20, verbose=1, batch_size=8)

AttributeError: 'Model' object has no attribute '_collected_trainable_weights'

In [ ]:
a_hats, b_hats = model.predict(X, batch_size=8)

In [ ]:
print(np.sum(a_hats[0, :, 0, 0]))
print(np.sum(a_hats[0, 0, :, 0]))
print(np.sum(a_hats[0, 0, 0, :]))

In [ ]:
a_hats = np.argmax(a_hats, axis=3)
b_hats = np.argmax(b_hats, axis=3)

In [ ]:
# Real Image
L = X[0].reshape((SIZE, SIZE)) * 255
a = y[0, :, :, 0] * 255
b = y[0, :, :, 1] * 255
lab = np.array([L, a, b]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('001.jpeg', img)

In [ ]:
# Real Image Binned
L = X[0].reshape((SIZE, SIZE)) * 255
a2 = y_binned[0, :, :, 0] * 8 + 4
b2 = y_binned[0, :, :, 1] * 8 + 4
lab = np.array([L, a2, b2]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('002.jpeg', img)

In [ ]:
L = X[0].reshape((SIZE, SIZE)) * 255
a_hat = a_hats[0, :, :] * 8 + 4
b_hat = b_hats[0, :, :] * 8 + 4
lab = np.array([L, a_hat, b_hat]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('003.jpeg', img)